In [ ]:
#install what needs to be installed setting up intance here

In [ ]:
#Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#set up file location here
import sys
sys.path.append('/content/gdrive/LLMHallucination')

In [ ]:
# import libraries
from Util import load_data_JSON, load_model
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [ ]:
#load the data
general_df,prompt_g,correct_g,gtp4_g,Info_g = load_data_JSON('input/general_data.json','general')
qa_df,prompt_q,correct_q,gtp4_q,Info_q = load_data_JSON('input/qa.json','qa')
sum_df,prompt_s,correct_s,gtp4_s,Info_s = load_data_JSON('input/summarization_data.json','sum')

In [ ]:
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf",access)

In [ ]:

def listOfTuples(l1, l2):
    return list(map(lambda x, y:(x,y), l1, l2)) 


In [ ]:
gen = pipeline("text-generation", model=model,torch_dtype = torch.float16, tokenizer=tokenizer)

In [ ]:
def get_response(prompt,max_len):
  sequences = gen(prompt,do_sample=True,top_k = 5, num_return_sequences =1, eos_token_id = tokenizer.eos_token_id, max_length = max_len +len(prompt) )
  return (sequences[0]['generated_text'])

In [ ]:
#run of the prompts 
out_q =[]
system_prompt = """<s>[INST] <<SYS>>
"You are a huallucination detector. You MUST determine if the provided answer contains hallucination or not for the question based on the provided Knowledge. The answer you provided MUST be \"Yes\" or \"No\""
<</SYS>>

"""
for i in range(200):
    prompt = system_prompt + f"\n\n#Knowledge#: " + {Info_q[i]}+"\n\n#Question#: " + {prompt_q[i]} + "\n#Answer#: " + response + "\n#Your Judgement#: [/INST]"
    response = get_response(prompt,256)
    response = response[len(prompt):]
    out_q.append(response)
